In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pathlib

In [ ]:
# view
def draw_pts(img, pts, colors):
    for pt, color in zip(pts, colors):
        img = cv2.circle(img, (pt[0],pt[1]), 12*2, (int(color[0]), int(color[1]), int(color[2])), -1)
    return img

def write_number(img, pts, colors):
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(pts)):
        img = cv2.putText(img, str(i), tuple(pts[i,:]), font, 4, 
                          (int(colors[i,0]), int(colors[i,1]), int(colors[i,2])), 10, cv2.LINE_AA)
    return img

def rand_color(length):
    return np.random.randint(100, 255, (length, 3))

def draw_line(img, lines_s, lines_f, colors):
    for line_s, line_f, color in zip(lines_s, lines_f, colors):
        img = cv2.line(img, tuple(line_s), tuple(line_f), (int(color[0]), int(color[1]), int(color[2])), 5)
    return img

In [ ]:
# 処理
def A_mat(pts1, pts2):
    A = np.zeros((len(pts1), 9))
    i = 0
    for pt1, pt2 in zip(pts1, pts2):
        x1, y1, x2, y2 = pt1[0], pt1[1], pt2[0], pt2[1]
        A[i, 0] = x1*x2
        A[i, 1] = x1*y2
        A[i, 2] = x1
        A[i, 3] = x2*y1
        A[i, 4] = y1*y2
        A[i, 5] = y1
        A[i, 6] = x2
        A[i, 7] = y2
        A[i, 8] = 1
        i += 1
    return A

def cal_F(A):
    # Vの転置の仕様
    U, s, V = np.linalg.svd(A, full_matrices=False)
    f_idx = np.argmin(np.sum(np.abs(np.dot(A, V.T)),axis=0))
    F = V[-1, :].reshape((3,3))
    #print(F)
    return F

def reconst_F(F):
    U, s, V = np.linalg.svd(F, full_matrices=False)
    s[2]=0
    return np.dot(np.dot(U, np.diag(s)),V)

def pts_add_one(pts):
    add_ones = np.ones((len(pts),1))
    return np.concatenate((pts, add_ones), axis=1)

def cal_lines(img_width, paras):
    lines_s = []
    lines_f = []
    for a,b,c in paras:
        start = tuple(map(int, [0, -(c/b)]))
        fin = tuple(map(int, [img_width, -(a/b)*img_width-(c/b)]))
        lines_s.append(start)
        lines_f.append(fin)
    return lines_s, lines_f

In [ ]:
def save_img(img, name):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite(name, img)

# 同じ物体上の点での8点アルゴリズム

In [ ]:
# 画像
c_left_img = cv2.imread('image/c_left.jpg')
c_left_img = cv2.cvtColor(c_left_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(c_left_img)
save_img(c_left_img, "made_image/c_left_img.png")

In [ ]:
c_right_img = cv2.imread('image/c_right.jpg')
c_right_img = cv2.cvtColor(c_right_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(c_right_img)
save_img(c_right_img,"made_image/c_right_img.png")

In [ ]:
# 8点，13点で計算．最後の三つは検証用
c_left_csv = np.loadtxt("csv/c_left.csv", delimiter=',').astype(np.uint32)
c_right_csv = np.loadtxt("csv/c_right.csv", delimiter=',').astype(np.uint32)
c_left_8 = c_left_csv[:8,:]
c_left_13 = c_left_csv[:13,:]
c_left_check = c_left_csv[13:,:]
c_right_8 = c_right_csv[:8,:]
c_right_13 = c_right_csv[:13,:]
c_right_check = c_right_csv[13:,:]

## 8点で計算

In [ ]:
# 8点の位置（左）
colors = rand_color(len(c_left_8))
pted_c_left = draw_pts(np.copy(c_left_img), c_left_8, colors)
pted_c_left = write_number(np.copy(pted_c_left), c_left_8, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_c_left)
save_img(pted_c_left,"made_image/8pted_c_left.png")

In [ ]:
pted_c_right = draw_pts(np.copy(c_right_img), c_right_8, colors)
pted_c_right = write_number(np.copy(pted_c_right), c_right_8, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_c_right)
save_img(pted_c_right, "made_image/8pted_c_right.png")

In [ ]:
# Fの計算
A = A_mat(c_left_8, c_right_8)
F = cal_F(A)
re_F = reconst_F(F)
F_det = np.linalg.det(re_F)
F_eig, v = np.linalg.eig(re_F)
print(F_det)
print("")
print(F_eig)
print("")
print(re_F)
np.savetxt('F_mat/c8_F_mat.csv', re_F, delimiter=',', fmt='%f')

In [ ]:
# m1 F m2 = 0 の確認
c_left_8_ones = pts_add_one(c_left_8)
c_right_8_ones = pts_add_one(c_right_8)
for i in range(8):
    print(np.dot(np.dot(c_left_8_ones[i],re_F),c_right_8_ones[i]))

In [ ]:
# エピポーララインの計算
img_width = c_left_img.shape[1]
epi_para_left = np.dot(c_left_8_ones, re_F)
epi_para_right = np.dot(c_right_8_ones, re_F.T)
r_img_epiline_s, r_img_epiline_f = cal_lines(img_width, epi_para_left)
l_img_epiline_s, l_img_epiline_f = cal_lines(img_width, epi_para_right)

In [ ]:
lined_c_left = draw_line(np.copy(pted_c_left), l_img_epiline_s, l_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_left)
save_img(lined_c_left, "made_image/8lined_c_left.png")

In [ ]:
lined_c_right = draw_line(np.copy(pted_c_right), r_img_epiline_s, r_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_right)
save_img(lined_c_right,"made_image/8lined_c_right.png")

In [ ]:
# 検証用エピポーラ線
c_left_check_ones = pts_add_one(c_left_check)
c_right_check_ones = pts_add_one(c_right_check)
epi_para_left_check = np.dot(c_left_check_ones, re_F)
epi_para_right_check = np.dot(c_right_check_ones, re_F.T)
r_epiline_check_s, r_epiline_check_f = cal_lines(img_width, epi_para_left_check)
l_epiline_check_s, l_epiline_check_f = cal_lines(img_width, epi_para_right_check)

In [ ]:
c_left_check

In [ ]:
lined_c_left_check = draw_line(np.copy(c_left_img), l_epiline_check_s, l_epiline_check_f, colors)
lined_c_left_check = draw_pts(np.copy(lined_c_left_check), c_left_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_left_check)
save_img(lined_c_left_check, "made_image/8lined_c_left_check.png")

In [ ]:
lined_c_right_check = draw_line(np.copy(c_right_img), r_epiline_check_s, r_epiline_check_f, colors)
lined_c_right_check = draw_pts(np.copy(lined_c_right_check), c_right_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_right_check)
save_img(lined_c_right_check, "made_image/8lined_c_right_check.png")

## 13点で計算

In [ ]:
# 13点の位置（左）
colors = rand_color(len(c_left_13))
pted_c_left = draw_pts(np.copy(c_left_img), c_left_13, colors)
pted_c_left = write_number(np.copy(pted_c_left), c_left_13, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_c_left)
save_img(pted_c_left, "made_image/13pted_c_left.png")

In [ ]:
pted_c_right = draw_pts(np.copy(c_right_img), c_right_13, colors)
pted_c_right = write_number(np.copy(pted_c_right), c_right_13, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_c_right)
save_img(pted_c_right, "made_image/13pted_c_right.png")

In [ ]:
# Fの計算
A = A_mat(c_left_13, c_right_13)
F = cal_F(A)
re_F = reconst_F(F)
F_det = np.linalg.det(re_F)
F_eig, v = np.linalg.eig(re_F)
print(F_det)
print("")
print(F_eig)
print("")
print(re_F)
np.savetxt('F_mat/c13_F_mat.csv', re_F, delimiter=',', fmt='%f')

In [ ]:
# m1 F m2 = 0 の確認
c_left_13_ones = pts_add_one(c_left_13)
c_right_13_ones = pts_add_one(c_right_13)
for i in range(13):
    print(np.dot(np.dot(c_left_13_ones[i],re_F),c_right_13_ones[i]))

In [ ]:
img_width = c_left_img.shape[1]
epi_para_left = np.dot(c_left_13_ones, re_F)
epi_para_right = np.dot(c_right_13_ones, re_F.T)
r_img_epiline_s, r_img_epiline_f = cal_lines(img_width, epi_para_left)
l_img_epiline_s, l_img_epiline_f = cal_lines(img_width, epi_para_right)

In [ ]:
lined_c_left = draw_line(np.copy(pted_c_left), l_img_epiline_s, l_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_left)
save_img(lined_c_left,"made_image/13lined_c_left.png")

In [ ]:
lined_c_right = draw_line(np.copy(pted_c_right), r_img_epiline_s, r_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_right)
save_img(lined_c_right,"made_image/13lined_c_right.png")

In [ ]:
# 検証用エピポーラ線
c_left_check_ones = pts_add_one(c_left_check)
c_right_check_ones = pts_add_one(c_right_check)
epi_para_left_check = np.dot(c_left_check_ones, re_F)
epi_para_right_check = np.dot(c_right_check_ones, re_F.T)
r_epiline_check_s, r_epiline_check_f = cal_lines(img_width, epi_para_left_check)
l_epiline_check_s, l_epiline_check_f = cal_lines(img_width, epi_para_right_check)

In [ ]:
lined_c_left_check = draw_line(np.copy(c_left_img), l_epiline_check_s, l_epiline_check_f, colors)
lined_c_left_check = draw_pts(np.copy(lined_c_left_check), c_left_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_left_check)
save_img(lined_c_left_check,"made_image/13lined_c_left_check.png")

In [ ]:
lined_c_right_check = draw_line(np.copy(c_right_img), r_epiline_check_s, r_epiline_check_f, colors)
lined_c_right_check = draw_pts(np.copy(lined_c_right_check), c_right_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_c_right_check)
save_img(lined_c_right_check,"made_image/13lined_c_right_check.png")

# 遠近

In [ ]:
# 画像
d_left_img = cv2.imread('image/d_left.jpg')
d_left_img = cv2.cvtColor(d_left_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(d_left_img)
save_img(d_left_img,"made_image/d_left_img.png")

In [ ]:
d_right_img = cv2.imread('image/d_right.jpg')
d_right_img = cv2.cvtColor(d_right_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(d_right_img)
save_img(d_right_img,"made_image/d_right_img.png")

In [ ]:
# 8点，13点で計算．最後の三つは検証用
d_left_csv = np.loadtxt("csv/d_left.csv", delimiter=',').astype(np.uint32)
d_right_csv = np.loadtxt("csv/d_right.csv", delimiter=',').astype(np.uint32)
d_left_8 = d_left_csv[:8,:]
d_left_13 = d_left_csv[:13,:]
d_left_check = d_left_csv[13:,:]
d_right_8 = d_right_csv[:8,:]
d_right_13 = d_right_csv[:13,:]
d_right_check = d_right_csv[13:,:]

## 8点で計算

In [ ]:
# 8点の位置（左）
colors = rand_color(len(d_left_8))
pted_d_left = draw_pts(np.copy(d_left_img), d_left_8, colors)
pted_d_left = write_number(np.copy(pted_d_left), d_left_8, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_d_left)
save_img(pted_d_left,"made_image/8pted_d_left.png")

In [ ]:
pted_d_right = draw_pts(np.copy(d_right_img), d_right_8, colors)
pted_d_right = write_number(np.copy(pted_d_right), d_right_8, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_d_right)
save_img(pted_d_right,"made_image/8pted_d_right.png")

In [ ]:
# Fの計算
A = A_mat(d_left_8, d_right_8)
F = cal_F(A)
re_F = reconst_F(F)
F_det = np.linalg.det(re_F)
F_eig, v = np.linalg.eig(re_F)
print(F_det)
print("")
print(F_eig)
print("")
print(re_F)
np.savetxt('F_mat/d8_F_mat.csv', re_F, delimiter=',', fmt='%.7e')

In [ ]:
# m1 F m2 = 0 の確認
d_left_8_ones = pts_add_one(d_left_8)
d_right_8_ones = pts_add_one(d_right_8)
for i in range(8):
    print(np.dot(np.dot(d_left_8_ones[i],re_F),d_right_8_ones[i]))

In [ ]:
# エピポーララインの計算
img_width = d_left_img.shape[1]
epi_para_left = np.dot(d_left_8_ones, re_F)
epi_para_right = np.dot(d_right_8_ones, re_F.T)
r_img_epiline_s, r_img_epiline_f = cal_lines(img_width, epi_para_left)
l_img_epiline_s, l_img_epiline_f = cal_lines(img_width, epi_para_right)

In [ ]:
lined_d_left = draw_line(np.copy(pted_d_left), l_img_epiline_s, l_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_left)
save_img(lined_d_left,"made_image/8lined_d_left.png")

In [ ]:
lined_d_right = draw_line(np.copy(pted_d_right), r_img_epiline_s, r_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_right)
save_img(lined_d_right,"made_image/8lined_d_right.png")

In [ ]:
# 検証用エピポーラ線
d_left_check_ones = pts_add_one(d_left_check)
d_right_check_ones = pts_add_one(d_right_check)
epi_para_left_check = np.dot(d_left_check_ones, re_F)
epi_para_right_check = np.dot(d_right_check_ones, re_F.T)
r_epiline_check_s, r_epiline_check_f = cal_lines(img_width, epi_para_left_check)
l_epiline_check_s, l_epiline_check_f = cal_lines(img_width, epi_para_right_check)

In [ ]:
lined_d_left_check = draw_line(np.copy(d_left_img), l_epiline_check_s, l_epiline_check_f, colors)
lined_d_left_check = draw_pts(np.copy(lined_d_left_check), d_left_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_left_check)
save_img(lined_d_left_check,"made_image/8lined_d_left_check.png")

In [ ]:
lined_d_right_check = draw_line(np.copy(d_right_img), r_epiline_check_s, r_epiline_check_f, colors)
lined_d_right_check = draw_pts(np.copy(lined_d_right_check), d_right_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_right_check)
save_img(lined_d_right_check,"made_image/8lined_d_right_check.png")

## 13点で検証

In [ ]:
# 13点の位置（左）
colors = rand_color(len(d_left_13))
pted_d_left = draw_pts(np.copy(d_left_img), d_left_13, colors)
pted_d_left = write_number(np.copy(pted_d_left), d_left_13, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_d_left)
save_img(pted_d_left,"made_image/13pted_d_left.png")

In [ ]:
pted_d_right = draw_pts(np.copy(d_right_img), d_right_13, colors)
pted_d_right = write_number(np.copy(pted_d_right), d_right_13, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(pted_d_right)
save_img(pted_d_right,"made_image/13pted_d_right.png")

In [ ]:
# Fの計算
A = A_mat(d_left_13, d_right_13)
F = cal_F(A)
re_F = reconst_F(F)
F_det = np.linalg.det(re_F)
F_eig, v = np.linalg.eig(re_F)
print(F_det)
print("")
print(F_eig)
print("")
print(re_F)
np.savetxt('F_mat/d13_F_mat.csv', re_F, delimiter=',', fmt='%.8e')

In [ ]:
# m1 F m2 = 0 の確認
d_left_13_ones = pts_add_one(d_left_13)
d_right_13_ones = pts_add_one(d_right_13)
for i in range(13):
    print(np.dot(np.dot(d_left_13_ones[i],re_F),d_right_13_ones[i]))

In [ ]:
img_width = d_left_img.shape[1]
epi_para_left = np.dot(d_left_13_ones, re_F)
epi_para_right = np.dot(d_right_13_ones, re_F.T)
r_img_epiline_s, r_img_epiline_f = cal_lines(img_width, epi_para_left)
l_img_epiline_s, l_img_epiline_f = cal_lines(img_width, epi_para_right)

In [ ]:
lined_d_left = draw_line(np.copy(pted_d_left), l_img_epiline_s, l_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_left)
save_img(lined_d_left, "made_image/13lined_d_left.png")

In [ ]:
lined_d_right = draw_line(np.copy(pted_d_right), r_img_epiline_s, r_img_epiline_f, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_right)
save_img(lined_d_right, "made_image/13lined_d_right.png")

In [ ]:
# 検証用エピポーラ線
d_left_check_ones = pts_add_one(d_left_check)
d_right_check_ones = pts_add_one(d_right_check)
epi_para_left_check = np.dot(d_left_check_ones, re_F)
epi_para_right_check = np.dot(d_right_check_ones, re_F.T)
r_epiline_check_s, r_epiline_check_f = cal_lines(img_width, epi_para_left_check)
l_epiline_check_s, l_epiline_check_f = cal_lines(img_width, epi_para_right_check)

In [ ]:
lined_d_left_check = draw_line(np.copy(d_left_img), l_epiline_check_s, l_epiline_check_f, colors)
lined_d_left_check = draw_pts(np.copy(lined_d_left_check), d_left_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_left_check)
save_img(lined_d_left_check,"made_image/13lined_d_left_check.png")

In [ ]:
lined_d_right_check = draw_line(np.copy(d_right_img), r_epiline_check_s, r_epiline_check_f, colors)
lined_d_right_check = draw_pts(np.copy(lined_d_right_check), d_right_check, colors)
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(lined_d_right_check)
save_img(lined_d_right_check,"made_image/13lined_d_right_check.png")